# import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.font_manager import FontProperties
import random
import os
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.metrics import accuracy_score, roc_auc_score,f1_score
from sklearn.ensemble import  AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVC,SVR
import joblib
from xgboost import XGBClassifier
import lightgbm as lgb
import pickle
import sklearn
import statsmodels.api as sm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.inspection import PartialDependenceDisplay

seed=42
np.random.seed(seed)
random.seed(seed)

import warnings
warnings.filterwarnings('ignore')
# 设置 Matplotlib 支持中文显示
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体为黑体
matplotlib.rcParams['axes.unicode_minus'] = False    # 解决保存图像时负号 '-' 显示为方块的问题

datapath='../data_d2'

In [2]:
#3-7-15的决策版

In [3]:
feature_names=['drug_time','cumi_flag','jia_flag','tang_flag','TZD_flag',"age","diabetic_duration",  "dbp", "bmi_v1",  "ALT", "LDL_c", "Cr",  "FPG", "CPO", "HbA1c"]

In [4]:
def read_and_fillna(df_valid):
    df_valid_1=df_valid[df_valid['type_no_class'] ==1]
    df_valid_2=df_valid[df_valid['type_no_class'] ==2]
    df_valid1=df_valid_1.iloc[:, -17:]
    df_valid2=df_valid_2.iloc[:, -17:]
    # df_valid1 = df_valid1.fillna(df_valid1.median())
    # df_valid2 = df_valid2.fillna(df_valid2.median())
    return df_valid_1,df_valid_2,df_valid1,df_valid2

In [18]:
# df_3_valid=pd.read_csv(os.path.join(datapath, 'df_3_valid.csv'))
df_6_valid=pd.read_csv(os.path.join(datapath, 'df_6_valid.csv'))
df_12_valid=pd.read_csv(os.path.join(datapath, 'df_12_valid.csv'))
# df_3_regression=pd.read_csv(os.path.join(datapath, 'df_3_regression.csv'))
df_6_regression=pd.read_csv(os.path.join(datapath, 'df_6_regression.csv'))
df_12_regression=pd.read_csv(os.path.join(datapath, 'df_12_regression.csv'))
# df_3_valid_1,df_3_valid_2,df_3_valid1,df_3_valid2=read_and_fillna(df_3_valid)
df_6_valid_1,df_6_valid_2,df_6_valid1,df_6_valid2=read_and_fillna(df_6_valid)
df_12_valid_1,df_12_valid_2,df_12_valid1,df_12_valid2=read_and_fillna(df_12_valid)
# df_3_regression_1,df_3_regression_2,df_3_regression1,df_3_regression2=read_and_fillna(df_3_regression)
df_6_regression_1,df_6_regression_2,df_6_regression1,df_6_regression2=read_and_fillna(df_6_regression)
df_12_regression_1,df_12_regression_2,df_12_regression1,df_12_regression2=read_and_fillna(df_12_regression)

In [19]:
def Ensemble(df,df_all_columns,model,folds,drug='GLP_valid'):
    df_result= df_all_columns.copy()
    data=df.copy()
    print(drug.split('_')[0],end=': ')

    Scaler=StandardScaler()
    model_lst=[]#用于存储每个折的模型
    a=time.time()
    auc_list = []
    feature_importances_list = []  # 用于存储每个折的特征重要性

    acc_list = []
    f1_score_list = []
    Precision_list = []
    Precision_num_list = []#记录预测为1的数量

    for i in range(10):
        test_hosp_ids = folds[i]
        train_hosp_ids = []
        for j in range(10):
            if j!=i:
                train_hosp_ids.extend(folds[j])

        train_data = data[data['hosp_id_base'].isin(train_hosp_ids)]
        test_data = data[data['hosp_id_base'].isin(test_hosp_ids)]
        # 训练和测试数据是最后16列
        # X_train = train_data.iloc[:, -15:]
        # X_test = test_data.iloc[:, -15:]
        
        X_train = train_data.iloc[:, -16:]
        X_test = test_data.iloc[:, -16:]

        # 假设最后一列是标签
        y_train = X_train.pop(X_train.columns[-1])
        y_test = X_test.pop(X_test.columns[-1])
        X_train.columns = feature_names
        Scaler.fit(X_train)
        X_train=Scaler.transform(X_train)
        model.fit(X_train, y_train)
        model_lst.append(model)
        # y_pred = model.predict(X_test)

        # 输出概率预测
        X_test.columns = feature_names
        X_test=Scaler.transform(X_test)
        
        y_pred_proba = model.predict_proba(X_test)
        y_pred = (y_pred_proba[:, 1] >= 0.5).astype(int)
        # 将本折的预测结果存入df
        df_result.loc[test_data.index, drug] = y_pred

        # 计算准确率
        acc_list.append(accuracy_score(y_test, y_pred))
        # 计算f1_score
        f1_score_list.append(f1_score(y_test, y_pred))
        # 计算预测为1时，实际为1的概率，Precision
        Precision = np.sum(y_pred[y_test == 1] == 1) / np.sum(y_pred == 1)
        Precision_list.append(Precision)
        Precision_num_list.append(np.sum(y_pred == 1))

        # 对概率预测计算AUC
        roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])
        auc_list.append(roc_auc)
        
    #输出平均准确率，保留4位小数
    print(f'AUC: {np.mean(auc_list):.4f}',f'acc: {np.mean(acc_list):.4f}',f'f1_score: {np.mean(f1_score_list):.4f}',f'Precision: {np.mean(Precision_list):.4f}'
          ,f'预测为药物有效的比例: {np.mean(Precision_num_list)*10/data.shape[0]:.4f}',f'time: {time.time()-a:.4f}')
    # print(acc_list)
    
    #使用全部数据训练模型
    X = data.iloc[:, -16:-1]
    X.columns = feature_names
    y = data.iloc[:, -1]
    Scaler.fit(X)
    X_scaled = Scaler.transform(X)
    model.fit(X_scaled, y)
    model_name=str(model.__class__.__name__)
    path=os.path.join('../Deliver_v1/','checkpoints_d2',str(model_name))
    if not os.path.exists(path):
        os.makedirs(path)
    joblib.dump(model, os.path.join(path,drug+'_best_model.pkl'))
    joblib.dump(Scaler, os.path.join(path,drug+'_scaler.pkl'))
    return df_result

In [20]:
def Combined_test_valid(df,df_all_columns, model,scaler,drug='GLP_valid'):
    df_result = df_all_columns.copy()
    data = df.copy()
    X_test = data .iloc[:, -16:]
    y_test = X_test.pop(X_test.columns[-1])
    # X_test = X_test.fillna(X_test.median())
    X_test.columns = feature_names
    X_test_scaled = scaler.transform(X_test)
    y_pred=model.predict(X_test_scaled).astype(int)
    df_result.loc[:, drug] = y_pred
    return df_result
def Combined_test_regression(df,df_all_columns, model,scaler,scaler_y,drug='GLP_regression'):
    df_result = df_all_columns.copy()
    data = df.copy()
    X_test = data .iloc[:, -16:]
    y_test = X_test.pop(X_test.columns[-1])
    # X_test = X_test.fillna(X_test.median())
    X_test.columns = feature_names
    X_test_scaled = scaler.transform(X_test)
    y_pred=model.predict(X_test_scaled)
    y_pred=scaler_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)
    df_result.loc[:, drug] = y_pred
    return df_result
def mape(predict, target):
    # 先找到所有非零的位置
    non_zero = target != 0
    # 初始化 mape_value 数组
    mape_value = np.zeros_like(target)
    # 只在非零位置上计算 MAPE
    mape_value[non_zero] = np.abs((target[non_zero] - predict[non_zero]) / target[non_zero])
    return np.mean(mape_value)


def smape(predict, target):
    denominator = np.abs(target) + np.abs(predict)
    smape_value = np.where(denominator != 0, 2 * np.abs(target - predict) / denominator, 0)
    return np.mean(smape_value)

In [21]:
def Ensemble_r(df,df_all_columns,model,folds,drug='GLP_regression'):
    df_result= df_all_columns.copy()
    data=df.copy()
    print(drug.split('_')[0],end=': ')
    Scaler = StandardScaler()
    Scaler_y = StandardScaler()
    model_lst=[]#用于存储每个折的模型
    data_X=data.iloc[:,1 :-1]
    # print(data_X.columns)
    data_y=data.iloc[:, -1]
    a=time.time()
    r2_list = []
    rmse_list = []
    feature_importances_list = []  # 用于存储每个折的特征重要性
    
    y_test_all = np.array([])  # 初始化用于累积所有k折的真实值的数组
    y_pred_all = np.array([])  # 初始化用于累积所有k折的预测值的数组
    
    for i in range(10):
        test_hosp_ids = folds[i]
        train_hosp_ids = []
        for j in range(10):
            if j != i:
                train_hosp_ids.extend(folds[j])

        train_data = data[data['hosp_id_base'].isin(train_hosp_ids)]
        test_data = data[data['hosp_id_base'].isin(test_hosp_ids)]

        # 训练和测试数据是最后16列
        X_train = train_data.iloc[:, -16:]
        X_test = test_data.iloc[:, -16:]

        # 假设最后一列是标签
        y_train = X_train.pop(X_train.columns[-1])
        y_test = X_test.pop(X_test.columns[-1])
        
        Scaler_y.fit(y_train.values.reshape(-1, 1))
        y_train=Scaler_y.transform(y_train.values.reshape(-1, 1)).reshape(-1)

        X_train.columns = feature_names
        Scaler.fit(X_train)
        X_train=Scaler.transform(X_train)
        
        model.fit(X_train, y_train)
        model_lst.append(model)
        X_test.columns = feature_names
        X_test=Scaler.transform(X_test)
        
        y_pred = model.predict(X_test)
        y_pred=Scaler_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)
        
        df_result.loc[test_data.index, drug] = y_pred

        # 累积真实值和预测值
        y_test_all = np.concatenate([y_test_all, y_test])
        y_pred_all = np.concatenate([y_pred_all, y_pred])

        # 计算R²
        r2_list.append(r2_score(y_test, y_pred))
        # 计算RMSE
        rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred)))


    # 在所有k折交叉验证完成后计算校准斜率
    calibration_model = LinearRegression()
    # 重新塑形y_pred_all和y_test_all以适应fit函数的需要
    y_pred_reshaped = y_pred_all.reshape(-1, 1)
    y_test_reshaped = y_test_all.reshape(-1, 1)
    calibration_model.fit(y_pred_reshaped, y_test_reshaped)
    slope = calibration_model.coef_[0]


    # 输出平均R²和RMSE
    print(f'校准斜率（Calibration Slope）: {slope[0]:.4f}',f'R²: {np.mean(r2_list):.4f}', f'RMSE: {np.mean(rmse_list):.4f}'
          ,f'MAPE: {mape(y_pred_all, y_test_all):.4f}', f'SMAPE: {smape(y_pred_all, y_test_all):.4f}',f'time: {time.time()-a:.4f}')
    
    #使用全部数据训练模型
    X = data.iloc[:, -16:-1]
    X.columns = feature_names
    y = data.iloc[:, -1]
    Scaler.fit(X)
    X_scaled = Scaler.transform(X)
    Scaler_y.fit(y.values.reshape(-1, 1))
    y_scaled = Scaler_y.transform(y.values.reshape(-1, 1)).reshape(-1)
    model.fit(X_scaled, y_scaled)
    model_name=str(model.__class__.__name__)
    path=os.path.join('../Deliver_v1/','checkpoints_d2',str(model_name))
    if not os.path.exists(path):
        os.makedirs(path)
    joblib.dump(model, os.path.join(path,drug+'_best_model.pkl'))
    joblib.dump(Scaler, os.path.join(path,drug+'_scaler.pkl'))
    joblib.dump(Scaler_y, os.path.join(path,drug+'_scaler_y.pkl'))
    return df_result

In [22]:
#读取按照医院划分后的k折
def load_fold(path):
    with open(path, 'rb') as f:
        folds1 = pickle.load(f)
        for i in range(10):
            folds1[i]=np.array(folds1[i])
    return folds1
# folds_3_valid_GLP=load_fold(os.path.join(datapath, 'ten_folds/GLP_3_valid.pkl'))
folds_6_valid_GLP=load_fold(os.path.join(datapath, 'ten_folds/GLP_6_valid.pkl'))
folds_12_valid_GLP=load_fold(os.path.join(datapath, 'ten_folds/GLP_12_valid.pkl'))
# folds_3_regression_GLP=load_fold(os.path.join(datapath, 'ten_folds/GLP_3_regression.pkl'))
folds_6_regression_GLP=load_fold(os.path.join(datapath, 'ten_folds/GLP_6_regression.pkl'))
folds_12_regression_GLP=load_fold(os.path.join(datapath, 'ten_folds/GLP_12_regression.pkl'))
# folds_3_valid_SGLT=load_fold(os.path.join(datapath, 'ten_folds/SGLT_3_valid.pkl'))
folds_6_valid_SGLT=load_fold(os.path.join(datapath, 'ten_folds/SGLT_6_valid.pkl'))
folds_12_valid_SGLT=load_fold(os.path.join(datapath, 'ten_folds/SGLT_12_valid.pkl'))
# folds_3_regression_SGLT=load_fold(os.path.join(datapath, 'ten_folds/SGLT_3_regression.pkl'))
folds_6_regression_SGLT=load_fold(os.path.join(datapath, 'ten_folds/SGLT_6_regression.pkl'))
folds_12_regression_SGLT=load_fold(os.path.join(datapath, 'ten_folds/SGLT_12_regression.pkl'))

In [23]:
lr_c = LogisticRegression()
rf_c = RandomForestClassifier(n_estimators=30, random_state=42, n_jobs=-1)
svm_c = SVC(probability=True, random_state=42)
gb_c = GradientBoostingClassifier(random_state=42)
xgb_c = XGBClassifier(n_estimators=100, learning_rate=0.1, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7, random_state=42)
lgb_c = lgb.LGBMClassifier(objective='binary', num_leaves=31, learning_rate=0.05, n_estimators=20,force_col_wise=True, random_state=42, verbose=-1)

lr_r = LinearRegression()
rf_r = RandomForestRegressor(n_estimators=30, random_state=42)
svm_r=SVR()
gb_r = GradientBoostingRegressor(random_state=42)
xgb_r = XGBRegressor(n_estimators=100, learning_rate=0.1, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=7,random_state=42)
lgb_r = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.05, n_estimators=20,force_col_wise=True, random_state=42, verbose=-1)
# 禁止lgb输出info等
os.environ['LIGHTGBM_LOGGER'] = 'None'
lgb_r.set_params(silent=True)


LGBMRegressor(force_col_wise=True, learning_rate=0.05, n_estimators=20,
              objective='regression', random_state=42, silent=True, verbose=-1)

In [24]:
#使用调参后的模型
# model_3_GLP_valid=GradientBoostingClassifier(n_estimators=15,learning_rate=0.1,max_depth=2,random_state=42)
# model_3_SGLT_valid=GradientBoostingClassifier(n_estimators=30,learning_rate=0.1,max_depth=2,random_state=42)
# model_3_GLP_regression=GradientBoostingRegressor(n_estimators=30,max_depth=2,min_samples_leaf=3,random_state=42)
# model_3_SGLT_regression=GradientBoostingRegressor(n_estimators=40,max_depth=3,min_samples_leaf=4,random_state=42)

model_6_GLP_valid=GradientBoostingClassifier(n_estimators=20,learning_rate=0.1,max_depth=4,random_state=42)
model_6_SGLT_valid=GradientBoostingClassifier(n_estimators=25,learning_rate=0.2,max_depth=3,random_state=42)
model_6_GLP_regression=GradientBoostingRegressor(n_estimators=40,max_depth=3,min_samples_leaf=6,random_state=42)
model_6_SGLT_regression=GradientBoostingRegressor(n_estimators=60,max_depth=3,min_samples_leaf=6,random_state=42)

model_12_GLP_valid=GradientBoostingClassifier(n_estimators=30,learning_rate=0.1,max_depth=2,random_state=42)
model_12_SGLT_valid=GradientBoostingClassifier(n_estimators=30,learning_rate=0.1,max_depth=3,random_state=42)
model_12_GLP_regression=GradientBoostingRegressor(n_estimators=60,max_depth=2,min_samples_leaf=5,random_state=42)
model_12_SGLT_regression=GradientBoostingRegressor(n_estimators=50,max_depth=3,min_samples_leaf=3,random_state=42)

In [27]:
#更新训练数据后，未调参
print('6月预测模型：')
df_6_valid_GLP1=Ensemble(df_6_valid1,df_6_valid_1, model_6_GLP_valid,folds_6_valid_GLP,drug='GLP_6_valid')
df_6_valid_SGLT1=Ensemble(df_6_valid2,df_6_valid_2, model_6_SGLT_valid,folds_6_valid_SGLT,drug='SGLT_6_valid')
df_6_regression_GLP1=Ensemble_r(df_6_regression1,df_6_regression_1, model_6_GLP_regression,folds_6_regression_GLP,drug='GLP_6_regression')
df_6_regression_SGLT1=Ensemble_r(df_6_regression2,df_6_regression_2, model_6_SGLT_regression,folds_6_regression_SGLT,drug='SGLT_6_regression')

print('12月预测模型：')
df_12_valid_GLP1=Ensemble(df_12_valid1,df_12_valid_1, model_12_GLP_valid,folds_12_valid_GLP,drug='GLP_12_valid')
df_12_valid_SGLT1=Ensemble(df_12_valid2,df_12_valid_2, model_12_SGLT_valid,folds_12_valid_SGLT,drug='SGLT_12_valid')
df_12_regression_GLP1=Ensemble_r(df_12_regression1,df_12_regression_1, model_12_GLP_regression,folds_12_regression_GLP,drug='GLP_12_regression')
df_12_regression_SGLT1=Ensemble_r(df_12_regression2,df_12_regression_2, model_12_SGLT_regression,folds_12_regression_SGLT,drug='SGLT_12_regression')
print()
print('--------------------------------------------------')

6月预测模型：
GLP: AUC: 0.7574 acc: 0.6912 f1_score: 0.7060 Precision: 0.7068 预测为药物有效的比例: 0.5304 time: 5.0724
SGLT: AUC: 0.7308 acc: 0.6778 f1_score: 0.5616 Precision: 0.6607 预测为药物有效的比例: 0.3141 time: 10.5333
GLP: 校准斜率（Calibration Slope）: 1.0450 R²: 0.2366 RMSE: 1.2832 MAPE: 0.1292 SMAPE: 0.1281 time: 6.6220
SGLT: 校准斜率（Calibration Slope）: 1.0323 R²: 0.2132 RMSE: 1.2499 MAPE: 0.1216 SMAPE: 0.1202 time: 23.7048
12月预测模型：
GLP: AUC: 0.7606 acc: 0.6973 f1_score: 0.6601 Precision: 0.7056 预测为药物有效的比例: 0.4204 time: 3.2521
SGLT: AUC: 0.7384 acc: 0.6879 f1_score: 0.5664 Precision: 0.6713 预测为药物有效的比例: 0.3057 time: 12.7154
GLP: 校准斜率（Calibration Slope）: 1.0597 R²: 0.2035 RMSE: 1.4990 MAPE: 0.1478 SMAPE: 0.1452 time: 5.8638
SGLT: 校准斜率（Calibration Slope）: 1.0378 R²: 0.1712 RMSE: 1.4288 MAPE: 0.1344 SMAPE: 0.1323 time: 19.9795

--------------------------------------------------


In [12]:

# print('3月预测模型：')

print('6月预测模型：')
df_6_valid_GLP1=Ensemble(df_6_valid1,df_6_valid_1, model_6_GLP_valid,folds_6_valid_GLP,drug='GLP_6_valid')
df_6_valid_SGLT1=Ensemble(df_6_valid2,df_6_valid_2, model_6_SGLT_valid,folds_6_valid_SGLT,drug='SGLT_6_valid')
df_6_regression_GLP1=Ensemble_r(df_6_regression1,df_6_regression_1, model_6_GLP_regression,folds_6_regression_GLP,drug='GLP_6_regression')
df_6_regression_SGLT1=Ensemble_r(df_6_regression2,df_6_regression_2, model_6_SGLT_regression,folds_6_regression_SGLT,drug='SGLT_6_regression')

print('12月预测模型：')
df_12_valid_GLP1=Ensemble(df_12_valid1,df_12_valid_1, model_12_GLP_valid,folds_12_valid_GLP,drug='GLP_12_valid')
df_12_valid_SGLT1=Ensemble(df_12_valid2,df_12_valid_2, model_12_SGLT_valid,folds_12_valid_SGLT,drug='SGLT_12_valid')
df_12_regression_GLP1=Ensemble_r(df_12_regression1,df_12_regression_1, model_12_GLP_regression,folds_12_regression_GLP,drug='GLP_12_regression')
df_12_regression_SGLT1=Ensemble_r(df_12_regression2,df_12_regression_2, model_12_SGLT_regression,folds_12_regression_SGLT,drug='SGLT_12_regression')
print()
print('--------------------------------------------------')

6月预测模型：
GLP: AUC: 0.7597 acc: 0.6949 f1_score: 0.7136 Precision: 0.7136 预测为药物有效的比例: 0.5397 time: 5.7678
SGLT: AUC: 0.7341 acc: 0.6798 f1_score: 0.5819 Precision: 0.6644 预测为药物有效的比例: 0.3399 time: 11.6681
GLP: 校准斜率（Calibration Slope）: 1.0603 R²: 0.2490 RMSE: 1.2494 MAPE: 0.1266 SMAPE: 0.1254 time: 8.0494
SGLT: 校准斜率（Calibration Slope）: 1.0440 R²: 0.2179 RMSE: 1.2267 MAPE: 0.1196 SMAPE: 0.1182 time: 27.2442
12月预测模型：
GLP: AUC: 0.7550 acc: 0.6917 f1_score: 0.6684 Precision: 0.7017 预测为药物有效的比例: 0.4454 time: 3.7311
SGLT: AUC: 0.7433 acc: 0.6897 f1_score: 0.5858 Precision: 0.6896 预测为药物有效的比例: 0.3202 time: 13.9903
GLP: 校准斜率（Calibration Slope）: 1.0843 R²: 0.2067 RMSE: 1.4889 MAPE: 0.1472 SMAPE: 0.1446 time: 6.5009
SGLT: 校准斜率（Calibration Slope）: 1.0473 R²: 0.1814 RMSE: 1.4140 MAPE: 0.1331 SMAPE: 0.1312 time: 21.9211

--------------------------------------------------


In [13]:
path_v=os.path.join('../Deliver_v1/','checkpoints_d2','GradientBoostingClassifier')
path_r=os.path.join('../Deliver_v1/','checkpoints_d2','GradientBoostingRegressor')

#调用训练好的模型
# GLP_3_valid_model=joblib.load(os.path.join(path_v,'GLP_3_valid_best_model.pkl'))
# SGLT_3_valid_model=joblib.load(os.path.join(path_v,'SGLT_3_valid_best_model.pkl'))
# GLP_3_regression_model=joblib.load(os.path.join(path_r,'GLP_3_regression_best_model.pkl'))
# SGLT_3_regression_model=joblib.load(os.path.join(path_r,'SGLT_3_regression_best_model.pkl'))

GLP_6_valid_model=joblib.load(os.path.join(path_v,'GLP_6_valid_best_model.pkl'))
SGLT_6_valid_model=joblib.load(os.path.join(path_v,'SGLT_6_valid_best_model.pkl'))
GLP_6_regression_model=joblib.load(os.path.join(path_r,'GLP_6_regression_best_model.pkl'))
SGLT_6_regression_model=joblib.load(os.path.join(path_r,'SGLT_6_regression_best_model.pkl'))

GLP_12_valid_model=joblib.load(os.path.join(path_v,'GLP_12_valid_best_model.pkl'))
SGLT_12_valid_model=joblib.load(os.path.join(path_v,'SGLT_12_valid_best_model.pkl'))
GLP_12_regression_model=joblib.load(os.path.join(path_r,'GLP_12_regression_best_model.pkl'))
SGLT_12_regression_model=joblib.load(os.path.join(path_r,'SGLT_12_regression_best_model.pkl'))

#调用保存的scaler
# GLP_3_valid_scaler=joblib.load(os.path.join(path_v,'GLP_3_valid_scaler.pkl'))
# SGLT_3_valid_scaler=joblib.load(os.path.join(path_v,'SGLT_3_valid_scaler.pkl'))
# GLP_3_regression_scaler=joblib.load(os.path.join(path_r,'GLP_3_regression_scaler.pkl'))
# SGLT_3_regression_scaler=joblib.load(os.path.join(path_r,'SGLT_3_regression_scaler.pkl'))
# GLP_3_regression_scaler_y=joblib.load(os.path.join(path_r,'GLP_3_regression_scaler_y.pkl'))
# SGLT_3_regression_scaler_y=joblib.load(os.path.join(path_r,'SGLT_3_regression_scaler_y.pkl'))

GLP_6_valid_scaler=joblib.load(os.path.join(path_v,'GLP_6_valid_scaler.pkl'))
SGLT_6_valid_scaler=joblib.load(os.path.join(path_v,'SGLT_6_valid_scaler.pkl'))
GLP_6_regression_scaler=joblib.load(os.path.join(path_r,'GLP_6_regression_scaler.pkl'))
SGLT_6_regression_scaler=joblib.load(os.path.join(path_r,'SGLT_6_regression_scaler.pkl'))
GLP_6_regression_scaler_y=joblib.load(os.path.join(path_r,'GLP_6_regression_scaler_y.pkl'))
SGLT_6_regression_scaler_y=joblib.load(os.path.join(path_r,'SGLT_6_regression_scaler_y.pkl'))

GLP_12_valid_scaler=joblib.load(os.path.join(path_v,'GLP_12_valid_scaler.pkl'))
SGLT_12_valid_scaler=joblib.load(os.path.join(path_v,'SGLT_12_valid_scaler.pkl'))
GLP_12_regression_scaler=joblib.load(os.path.join(path_r,'GLP_12_regression_scaler.pkl'))
SGLT_12_regression_scaler=joblib.load(os.path.join(path_r,'SGLT_12_regression_scaler.pkl'))
GLP_12_regression_scaler_y=joblib.load(os.path.join(path_r,'GLP_12_regression_scaler_y.pkl'))
SGLT_12_regression_scaler_y=joblib.load(os.path.join(path_r,'SGLT_12_regression_scaler_y.pkl'))

In [14]:
def Combined_all_test(df_lst,df_all_columns_lst,model,scaler,scaler_y,drug='GLP_valid'):
    df_all_result=pd.DataFrame()
    flag=drug.split('_')[-1]
    for i,df,df_all_columns in zip(range(len(df_lst)),df_lst,df_all_columns_lst):
        df_result= df_all_columns.copy()
        data=df.copy()

        X_test = data.iloc[:, -16:]
        drug_time=92 if i<=1 else 183 if i<=3 else 365
        X_test.iloc[:,0]=drug_time
        y_test = X_test.pop(X_test.columns[-1])
        # X_test=X_test.fillna(X_test.median())
        X_test.columns=feature_names
        X_test_scaled = scaler.transform(X_test)
        if flag=='valid':
            y_pred=model.predict(X_test_scaled).astype(int) 
        else:#regression
            y_pred = model.predict(X_test_scaled)
            y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)
        
        df_result.loc[:, drug]=y_pred
        df_all_result=pd.concat([df_all_result,df_result],axis=0)
    df_all_result=df_all_result[['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date',drug]]
    df_all_result.sort_values(by=['user_id',  'type_no_class','first_Hb_date'], inplace=True)   
    print(df_all_result.shape)
    return df_all_result
df_valid_lst=[df_6_valid1,df_6_valid2,df_12_valid1,df_12_valid2]
df_all_columns_valid_lst=[df_6_valid_1,df_6_valid_2,df_12_valid_1,df_12_valid_2]
df_regression_lst=[df_6_regression1,df_6_regression2,df_12_regression1,df_12_regression2]
df_all_columns_regression_lst=[df_6_regression_1,df_6_regression_2,df_12_regression_1,df_12_regression_2]

In [15]:
df_6_valid_GLP=Combined_all_test(df_valid_lst,df_all_columns_valid_lst,GLP_6_valid_model,GLP_6_valid_scaler,GLP_6_regression_scaler_y,drug='GLP_6_valid')
df_6_valid_SGLT=Combined_all_test(df_valid_lst,df_all_columns_valid_lst,SGLT_6_valid_model,SGLT_6_valid_scaler,SGLT_6_regression_scaler_y,drug='SGLT_6_valid')
df_6_regression_GLP=Combined_all_test(df_regression_lst,df_all_columns_regression_lst,GLP_6_regression_model,GLP_6_regression_scaler,GLP_6_regression_scaler_y,drug='GLP_6_regression')
df_6_regression_SGLT=Combined_all_test(df_regression_lst,df_all_columns_regression_lst,SGLT_6_regression_model,SGLT_6_regression_scaler,SGLT_6_regression_scaler_y,drug='SGLT_6_regression')
#合并4列预测结果
df_6_valid_final = pd.merge(df_6_valid_GLP, df_6_valid_SGLT, on=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date'], how='inner')
df_6_regression_final = pd.merge(df_6_regression_GLP, df_6_regression_SGLT, on=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date'], how='inner')
df_6_final= pd.merge(df_6_valid_final, df_6_regression_final, on=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date'], how='inner')
#筛去因指定用药时间后，结果重复的样本,但注意保留指定过不同用药时长的患者
# df_6_final.drop_duplicates(subset=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','GLP_6_regression','SGLT_6_regression'],keep='first', inplace=True)
print(df_6_final.shape)
df_6_final.head()

(37464, 6)
(37464, 6)
(38683, 6)
(38683, 6)
(37464, 9)


,user_id,hosp_id_base,type_no_class,first_Hb_date,target_Hb_date,GLP_6_valid,SGLT_6_valid,GLP_6_regression,SGLT_6_regression
0,10238.0,20006.0,1.0,2019-10-24,2020-07-24,1,1,7.250586,6.979465
1,902628.0,20006.0,2.0,2018-12-17,2019-03-26,0,0,7.591098,7.623802
2,902628.0,20006.0,2.0,2018-12-17,2019-06-25,0,0,7.591098,7.623802
3,985157.0,20006.0,1.0,2019-05-07,2019-08-13,0,0,7.223747,7.331540
4,985248.0,20006.0,1.0,2016-10-31,2017-02-17,0,0,7.044863,7.215858


In [16]:
df_12_valid_GLP=Combined_all_test(df_valid_lst,df_all_columns_valid_lst,GLP_12_valid_model,GLP_12_valid_scaler,GLP_12_regression_scaler_y,drug='GLP_12_valid')
df_12_valid_SGLT=Combined_all_test(df_valid_lst,df_all_columns_valid_lst,SGLT_12_valid_model,SGLT_12_valid_scaler,SGLT_12_regression_scaler_y,drug='SGLT_12_valid')
df_12_regression_GLP=Combined_all_test(df_regression_lst,df_all_columns_regression_lst,GLP_12_regression_model,GLP_12_regression_scaler,GLP_12_regression_scaler_y,drug='GLP_12_regression')
df_12_regression_SGLT=Combined_all_test(df_regression_lst,df_all_columns_regression_lst,SGLT_12_regression_model,SGLT_12_regression_scaler,SGLT_12_regression_scaler_y,drug='SGLT_12_regression')
#合并4列预测结果
df_12_valid_final = pd.merge(df_12_valid_GLP, df_12_valid_SGLT, on=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date'], how='inner')
df_12_regression_final = pd.merge(df_12_regression_GLP, df_12_regression_SGLT, on=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date'], how='inner')
df_12_final= pd.merge(df_12_valid_final, df_12_regression_final, on=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date'], how='inner')
#筛去因指定用药时间后，结果重复的样本,但注意保留指定过不同用药时长的患者
# df_12_final.drop_duplicates(subset=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','GLP_12_regression','SGLT_12_regression'],keep='first', inplace=True)
print(df_12_final.shape)
df_12_final.head()

(37464, 6)
(37464, 6)
(38683, 6)
(38683, 6)
(37464, 9)


,user_id,hosp_id_base,type_no_class,first_Hb_date,target_Hb_date,GLP_12_valid,SGLT_12_valid,GLP_12_regression,SGLT_12_regression
0,10238.0,20006.0,1.0,2019-10-24,2020-07-24,1,1,7.280212,7.207287
1,902628.0,20006.0,2.0,2018-12-17,2019-03-26,0,0,8.174397,8.311944
2,902628.0,20006.0,2.0,2018-12-17,2019-06-25,0,0,8.174397,8.311944
3,985157.0,20006.0,1.0,2019-05-07,2019-08-13,0,0,7.754273,7.643734
4,985248.0,20006.0,1.0,2016-10-31,2017-02-17,0,0,7.423391,7.378617


In [17]:
df_6_12_final = pd.merge(df_6_final, df_12_final, on=['user_id','hosp_id_base', 'type_no_class','first_Hb_date','target_Hb_date'], how='inner')
df_6_12_final1=df_6_12_final.drop(columns=['target_Hb_date'])
df_6_12_final1.drop_duplicates(keep='first', inplace=True)

df_6_12_final1.to_csv(os.path.join(datapath, 'df_6_12_final.csv'), index=False)
# df_3_6_12_final.head(30)

In [18]:
df_6_12_final1.shape

(33934, 12)

In [19]:
df_6_12_final1.head(10)

,user_id,hosp_id_base,type_no_class,first_Hb_date,GLP_6_valid,SGLT_6_valid,GLP_6_regression,SGLT_6_regression,GLP_12_valid,SGLT_12_valid,GLP_12_regression,SGLT_12_regression
0,10238.0,20006.0,1.0,2019-10-24,1,1,7.250586,6.979465,1,1,7.280212,7.207287
1,902628.0,20006.0,2.0,2018-12-17,0,0,7.591098,7.623802,0,0,8.174397,8.311944
3,985157.0,20006.0,1.0,2019-05-07,0,0,7.223747,7.331540,0,0,7.754273,7.643734
4,985248.0,20006.0,1.0,2016-10-31,0,0,7.044863,7.215858,0,0,7.423391,7.378617
5,1025718.0,20006.0,2.0,2021-04-14,1,0,7.312760,7.243799,1,0,7.353223,7.373058
6,1025718.0,20006.0,2.0,2021-04-14,0,0,7.211372,7.228332,1,0,7.353223,7.290009
7,1046523.0,20006.0,1.0,2020-06-08,1,1,6.899182,6.912368,1,1,7.162282,7.099627
8,1046523.0,20006.0,1.0,2020-06-08,1,1,6.899182,6.891884,1,1,7.162282,7.099627
9,1046523.0,20006.0,1.0,2020-06-08,1,1,7.062380,6.985210,1,1,7.162282,7.099627
11,1046523.0,20006.0,2.0,2018-07-17,1,1,6.574461,6.654237,1,1,7.155647,7.009215


In [2]:
def Decision(df,e1,g1,e2,g2,result):
    df=df.copy()
    def piecewise_function(x):
        if x < 0:
            return  x
        elif x<0.5:
            return 2 * x
        else:
            return 4 * x**2
    def calculate_score(e1,g1,e2,g2,W_coef=1,threshold_drug=0.15,threshold_score=0):
        score1 = W_coef * e1 - piecewise_function(g1-7)
        score2 = W_coef * e2 - piecewise_function(g2-7)
        if score1-score2>threshold_drug:
            #推荐1
            return 'GLP'
        elif score2-score1>threshold_drug:
            #推荐2
            return 'SGLT'
        else:
            if score1+score2>=threshold_score:
                return 'maybe both'
            else:
                return 'maybe neither'
    def get_score(e1,g1,e2,g2,W_coef=1):
        score1 = W_coef * e1 - piecewise_function(g1-7)
        score2 = W_coef * e2 - piecewise_function(g2-7)
        return score1,score2

    df['score_GLP'],df['score_SGLT']=zip(*df.apply(lambda row: get_score(row[e1], row[g1], row[e2], row[g2]), axis=1))
    df[result] = df.apply(lambda row: calculate_score(row[e1], row[g1], row[e2], row[g2]), axis=1)
    #统计推荐结果
    print(df[result].value_counts(normalize=True))
    return df

In [3]:
import pandas as pd
df_6_12_final=pd.read_csv(os.path.join(datapath, 'df_6_12_final.csv'))
df_6_12_final_only=df_6_12_final.copy()
df_6_12_final_only.drop_duplicates(subset=['user_id','hosp_id_base', 'type_no_class'],keep='first', inplace=True)
df_final_61=Decision(df_6_12_final_only,'GLP_6_valid','GLP_6_regression','SGLT_6_valid','SGLT_6_regression','recommend_6')
df_final_6=df_final_61[['user_id','GLP_6_valid','GLP_6_regression','SGLT_6_valid','SGLT_6_regression','recommend_6','score_GLP','score_SGLT']]
df_final_6.head(10)

recommend_6
GLP              0.567728
maybe both       0.173477
SGLT             0.169198
maybe neither    0.089597
Name: proportion, dtype: float64


,user_id,GLP_6_valid,GLP_6_regression,SGLT_6_valid,SGLT_6_regression,recommend_6,score_GLP,score_SGLT
0,10238.0,1,7.250586,1,6.979465,SGLT,0.498828,1.020535
1,902628.0,0,7.591098,0,7.623802,GLP,-1.397588,-1.556513
2,985157.0,0,7.223747,0,7.331540,GLP,-0.447494,-0.663080
3,985248.0,0,7.044863,0,7.215858,GLP,-0.089726,-0.431716
4,1025718.0,1,7.312760,0,7.243799,GLP,0.374481,-0.487599
6,1046523.0,1,6.899182,1,6.912368,maybe both,1.100818,1.087632
9,1046523.0,1,6.574461,1,6.654237,maybe both,1.425539,1.345763
12,1064020.0,1,6.891037,1,7.007622,maybe both,1.108963,0.984756
15,1064162.0,1,6.857082,1,6.893941,maybe both,1.142918,1.106059
16,1064162.0,1,6.664215,1,6.752191,maybe both,1.335785,1.247809


In [4]:
df_final_121=Decision(df_6_12_final_only,'GLP_12_valid','GLP_12_regression','SGLT_12_valid','SGLT_12_regression','recommend_12')
df_final_12=df_final_121[['user_id','GLP_12_valid','GLP_12_regression','SGLT_12_valid','SGLT_12_regression','recommend_12','score_GLP','score_SGLT']]
df_final_12.head(10)


recommend_12
GLP              0.495361
SGLT             0.270951
maybe both       0.161056
maybe neither    0.072632
Name: proportion, dtype: float64


,user_id,GLP_12_valid,GLP_12_regression,SGLT_12_valid,SGLT_12_regression,recommend_12,score_GLP,score_SGLT
0,10238.0,1,7.280212,1,7.207287,maybe both,0.439576,0.585426
1,902628.0,0,8.174397,0,8.311944,GLP,-5.516832,-6.884784
2,985157.0,0,7.754273,0,7.643734,SGLT,-2.275710,-1.657571
3,985248.0,0,7.423391,0,7.378617,maybe neither,-0.846782,-0.757235
4,1025718.0,1,7.353223,0,7.373058,GLP,0.293555,-0.746117
6,1046523.0,1,7.162282,1,7.099627,maybe both,0.675436,0.800745
9,1046523.0,1,7.155647,1,7.009215,SGLT,0.688706,0.981570
12,1064020.0,1,7.231959,1,7.151744,SGLT,0.536082,0.696511
15,1064162.0,1,7.112015,1,7.055164,maybe both,0.775970,0.889672
16,1064162.0,1,7.121199,1,6.913775,SGLT,0.757601,1.086225


In [23]:
df_final= df_final_6[['user_id']]
# df_final['recommend_3'] = df_final_3.iloc[:, -3]
df_final['recommend_6'] = df_final_6.iloc[:, -3]
df_final['recommend_12'] = df_final_12.iloc[:, -3]
df_final.head(30)

,user_id,recommend_6,recommend_12
0,10238.0,SGLT,maybe both
1,902628.0,maybe neither,GLP
2,985157.0,GLP,SGLT
3,985248.0,GLP,maybe neither
4,1025718.0,GLP,GLP
6,1046523.0,maybe both,maybe both
9,1046523.0,maybe both,SGLT
12,1064020.0,maybe both,maybe both
15,1064162.0,maybe both,maybe both
16,1064162.0,maybe both,SGLT


In [60]:
def classify_predictions(row):
    unique_values = set(row)
    # Check for '三个一致'
    if len(unique_values) == 1:
        if 'GLP' in unique_values:
            return 'GLP(certain)'
        elif 'SGLT' in unique_values:
            return 'SGLT(certain)'
        elif 'maybe both' in unique_values:
            return 'both(certain)'
        else:
            return 'neither(certain)'

    if len(unique_values) == 2:
        if unique_values == {'GLP', 'SGLT'} :
            if row['recommend_3'] == row['recommend_12'] and row['recommend_3'] != row['recommend_6']:
                #二夹一，推荐多数的
                return row[0]
            else:#渐变型
                if row['recommend_3'] ==row['recommend_6']:
                    return '短期'+row['recommend_3']
                else:
                    return '长期'+row['recommend_12']
        elif unique_values == {'maybe both', 'maybe neither'}:
            return 'None'
        else:#一个确定，一个不确定，返回确定的值
            if 'GLP' in unique_values:
                return 'GLP'
            else:
                return 'SGLT'

    # Check for '三个不一致'
    else:
        if 'maybe neither' in unique_values:
            return 'None'
        else:
            if row['recommend_3'] !='SGLT':
                return 'SGLT'
            else:
                return 'GLP'


# Update the function to calculate distribution proportions including the new classification logic
def calculate_distribution_proportions(df):
    df=df.iloc[:,1:].copy()
    df['recommend'] = df.apply(classify_predictions, axis=1)

    # Calculate the proportions of each classification
    total_rows = df.shape[0]
    distribution_counts = df['recommend'].value_counts()
    distribution_proportion = distribution_counts / total_rows
    return df,distribution_proportion

In [61]:

df_final_time,distribution_proportion=calculate_distribution_proportions(df_final)
#以百分比形式输出，暴露2位小数
distribution_proportions = distribution_proportion.apply(lambda x: f'{x:.2%}')
print(distribution_proportions)
print()
distribution_proportion = distribution_proportion.apply(lambda x: round(x, 4))
print('GLP：',round(100*(distribution_proportion['GLP(certain)']+distribution_proportion['GLP']+distribution_proportion['短期GLP']+distribution_proportion['长期GLP']),2),'%')
print('SGLT：',round(100*(distribution_proportion['SGLT(certain)']+distribution_proportion['SGLT']+distribution_proportion['短期SGLT']+distribution_proportion['长期SGLT']),2),'%')
print('undetermined：',round(100*(distribution_proportion['both(certain)']+distribution_proportion['neither(certain)']+distribution_proportion['None']),2),'%')
df_final_time.head(30)

recommend
GLP                 28.92%
GLP(certain)        15.23%
短期GLP               12.76%
长期SGLT              12.65%
SGLT                12.64%
None                 9.63%
both(certain)        4.06%
SGLT(certain)        2.36%
长期GLP                0.92%
短期SGLT               0.45%
neither(certain)     0.40%
Name: count, dtype: object

GLP： 57.83 %
SGLT： 28.1 %
undetermined： 14.09 %


,recommend_3,recommend_6,recommend_12,recommend
0,GLP,SGLT,maybe both,SGLT
1,SGLT,GLP,GLP,长期GLP
3,GLP,GLP,maybe both,GLP
4,GLP,maybe neither,maybe neither,GLP
5,GLP,maybe neither,maybe neither,GLP
8,maybe neither,maybe neither,GLP,GLP
10,GLP,GLP,SGLT,短期GLP
12,SGLT,GLP,maybe neither,None
14,maybe both,maybe both,SGLT,SGLT
17,GLP,SGLT,SGLT,长期SGLT
